In [4]:
pip install --force-reinstall matplotlib pandas

  Using cached matplotlib-3.9.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
     ---------------------------------------- 0.0/165.5 kB ? eta -:--:--
     --------- --------------------------- 41.0/165.5 kB 991.0 kB/s eta 0:00:01
     -------------------------------------  163.8/165.5 kB 2.5 MB/s eta 0:00:01
     -------------------------------------- 165.5/165.5 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 3.2 MB/s eta 0:00:00
Using cached matplotlib-3.9.0-cp311-cp311-win_amd64.whl (8.0 MB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 2.4 MB/s eta 0:00:05
    --------------------------------------- 0.3/11.6 MB 2.8 MB/s eta 0:00:05
   - -------------------------------------- 0.4/11.6 MB 2.8 MB/s eta 0:00:05
   - -------------------------------------- 0.5/11.6 MB 2.8 MB/s eta 0:00:04
   -- -----------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires python-dateutil==2.8.2, but you have python-dateutil 2.9.0.post0 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 2.0.0 which is in

In [5]:
import numpy as np
import requests
import pandas as pd
import ast

2.0.0


In [2]:

api_token = '666a9c19073fb4.00690494'
from_date = "2020-01-01"
to_date = "2024-06-10"
url_template = 'https://eodhd.com/api/news?s=AAPL.US&offset={}&limit=1000&api_token={}&fmt=json&from={}&to={}'

def fetch_news(from_date, to_date, api_token):
    all_news = []
    offset = 0
    while True:
        url = url_template.format(offset, api_token, from_date, to_date)
        response = requests.get(url)
        if response.status_code != 200:
            print("Failed to fetch data:", response.status_code)
            break
        data = response.json()
        if not data:
            print("No more data available.")
            break
        all_news.extend(data)
        offset += 1000

    if all_news:
        df_news = pd.DataFrame(all_news)
        # Filtrar por fecha si la columna 'date' está presente
        if 'date' in df_news.columns:
            df_news['date'] = pd.to_datetime(df_news['date'])
            df_news = df_news[(df_news['date'] >= from_date) & (df_news['date'] <= to_date)]
        return df_news
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío si no hay datos

# Usar la función para obtener noticias
df_news = fetch_news(from_date, to_date, api_token)
print(df_news)

Failed to fetch data: 402
Empty DataFrame
Columns: []
Index: []


Me parecían pocas noticias en el 2020_2023 así que descargué todas otra vez desde la API ya que el csv solo me devolvía 9294 rows
df = pd.read_csv("Data/df_news_2020_2023.csv")

In [198]:
df_news = pd.read_csv(r".\data\primary\df_news_trading.csv")

In [199]:
df_news_model=df_news[['date','title']]
df_news_model.rename(columns={'title': 'sentence'}, inplace=True)

C:\Users\river\AppData\Local\Temp\ipykernel_7360\2286996986.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news_model.rename(columns={'title': 'sentence'}, inplace=True)


In [201]:
df_news_model=df_news_model[~df_news_model['sentence'].isna()]

In [195]:
new_df = df_news[['date', 'sentiment']]

In [8]:
new_df=new_df[~new_df['sentiment'].isna()]

In [10]:
new_df['polarity'] = new_df['sentiment'].apply(lambda x: ast.literal_eval(x)['polarity'])

In [11]:

# Función para determinar si el sentimiento es "Negative"
def is_negative(polarity):
    return 'negative' if polarity < -0.2 else ''

# Función para determinar si el sentimiento es "Positive"
def is_positive(polarity):
    return 'positive' if polarity > 0.3 else ''

# Función para determinar si el sentimiento es "Neutro"
def is_neutral(polarity):
    return 'neutro' if polarity >= -0.2 and polarity <= 0.3 else ''


# Aplicar las funciones para crear nuevas columnas
# Usamos un manejo de errores robusto para asegurarnos de que accedemos a 'polarity' de manera segura
new_df['Negative'] = new_df['polarity'].apply(lambda x: is_negative(x))
new_df['Positive'] = new_df['polarity'].apply(lambda x: is_positive(x))
new_df['Neutro'] = new_df['polarity'].apply(lambda x: is_neutral(x))

In [12]:
df_reduced = new_df

In [13]:
# Miramos como se balancea

count_negative = (df_reduced['Negative'] == 'negative').sum()
count_positive = (df_reduced['Positive'] == 'positive').sum()
count_neutro = (df_reduced['Neutro'] == 'neutro').sum()


print(f"Total Negative: {count_negative}")
print(f"Total Positive: {count_positive}")
print(f"Total Neutro: {count_neutro}")

Total Negative: 3682
Total Positive: 24808
Total Neutro: 4822


In [18]:
df_reduced

,date,sentiment,polarity,Negative,Positive,Neutro
0,2020-01-15T12:00:00+00:00,"{'polarity': -0.103, 'neg': 0.023, 'neu': 0.95...",-0.103,,,neutro
1,2020-01-27T11:55:00+00:00,"{'polarity': 0.924, 'neg': 0, 'neu': 0.948, 'p...",0.924,,positive,
2,2020-01-28T23:41:00+00:00,"{'polarity': 0.999, 'neg': 0.025, 'neu': 0.851...",0.999,,positive,
3,2020-01-30T22:00:00+00:00,"{'polarity': 0.972, 'neg': 0, 'neu': 0.914, 'p...",0.972,,positive,
4,2020-01-30T22:00:00+00:00,"{'polarity': 0.996, 'neg': 0.072, 'neu': 0.818...",0.996,,positive,
...,...,...,...,...,...,...
33322,2024-06-09 16:30:17+00:00,"{'polarity': 0.996, 'neg': 0, 'neu': 0.888, 'p...",0.996,,positive,
33323,2024-06-10 00:00:00+00:00,"{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}",0.000,,,neutro
33324,2024-06-10 00:00:00+00:00,"{'polarity': -0.318, 'neg': 0.173, 'neu': 0.82...",-0.318,negative,,
33325,2024-06-10 00:00:00+00:00,"{'polarity': 0.382, 'neg': 0, 'neu': 0.776, 'p...",0.382,,positive,


In [22]:
# Convertir la columna 'date' a datetime

df_reduced['date'] = pd.to_datetime(df_reduced['date'], format='ISO8601')

# Cambiar el formato de la columna 'date' a 'dd-mm-yyyy'
df_reduced['date'] = df_reduced['date'].dt.strftime('%Y-%m-%d')

# Mostrar el DataFrame para verificar los cambios
print(df_reduced)

             date                                          sentiment  \
0      2020-01-15  {'polarity': -0.103, 'neg': 0.023, 'neu': 0.95...   
1      2020-01-27  {'polarity': 0.924, 'neg': 0, 'neu': 0.948, 'p...   
2      2020-01-28  {'polarity': 0.999, 'neg': 0.025, 'neu': 0.851...   
3      2020-01-30  {'polarity': 0.972, 'neg': 0, 'neu': 0.914, 'p...   
4      2020-01-30  {'polarity': 0.996, 'neg': 0.072, 'neu': 0.818...   
...           ...                                                ...   
33322  2024-06-09  {'polarity': 0.996, 'neg': 0, 'neu': 0.888, 'p...   
33323  2024-06-10      {'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}   
33324  2024-06-10  {'polarity': -0.318, 'neg': 0.173, 'neu': 0.82...   
33325  2024-06-10  {'polarity': 0.382, 'neg': 0, 'neu': 0.776, 'p...   
33326  2024-06-10  {'polarity': 0.649, 'neg': 0, 'neu': 0.773, 'p...   

       polarity  Negative  Positive  Neutro  
0        -0.103                      neutro  
1         0.924            positive        

In [23]:
# Crear una función que determina la categoría basada en las columnas Negative, Positive, Neutro. O sea lo pasamos a una única columna para poder agrupar por días
def categorize(row):
    if row['Positive'] == 'positive':
        return 'Positive'
    elif row['Negative'] == 'negative':
        return 'Negative'
    elif row['Neutro'] == 'neutro':
        return 'Neutro'
    else:
        return 'Undefined'  # En caso de que ninguna categoría esté asignada

# Aplicar la función a cada fila
df_reduced['Category'] = df_reduced.apply(categorize, axis=1)

# Mostrar el resultado
print(df_reduced[['date', 'Category']])

             date  Category
0      2020-01-15    Neutro
1      2020-01-27  Positive
2      2020-01-28  Positive
3      2020-01-30  Positive
4      2020-01-30  Positive
...           ...       ...
33322  2024-06-09  Positive
33323  2024-06-10    Neutro
33324  2024-06-10  Negative
33325  2024-06-10  Positive
33326  2024-06-10  Positive

[33312 rows x 2 columns]


In [98]:
df_reduced.groupby('date')[['Negative','Positive', 'Neutro']].agg({'Negative': 'count', 'Positive': 'count', 'Neutro': 'count'})
df_reduced.groupby('date')[['Negative','Positive', 'Neutro']].agg({'Negative': 'count', 'Positive': 'count', 'Neutro': 'count'})


,Neutro
date,
2020-01-15,1
2020-01-27,1
2020-01-28,1
2020-01-30,2
2020-01-31,1
...,...
2024-06-06,25
2024-06-07,22
2024-06-08,3


In [99]:
df_reduced

,date,sentiment,polarity,Negative,Positive,Neutro,Category
0,2020-01-15,"{'polarity': -0.103, 'neg': 0.023, 'neu': 0.95...",-0.103,,,neutro,Neutro
1,2020-01-27,"{'polarity': 0.924, 'neg': 0, 'neu': 0.948, 'p...",0.924,,positive,,Positive
2,2020-01-28,"{'polarity': 0.999, 'neg': 0.025, 'neu': 0.851...",0.999,,positive,,Positive
3,2020-01-30,"{'polarity': 0.972, 'neg': 0, 'neu': 0.914, 'p...",0.972,,positive,,Positive
4,2020-01-30,"{'polarity': 0.996, 'neg': 0.072, 'neu': 0.818...",0.996,,positive,,Positive
...,...,...,...,...,...,...,...
33322,2024-06-09,"{'polarity': 0.996, 'neg': 0, 'neu': 0.888, 'p...",0.996,,positive,,Positive
33323,2024-06-10,"{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}",0.000,,,neutro,Neutro
33324,2024-06-10,"{'polarity': -0.318, 'neg': 0.173, 'neu': 0.82...",-0.318,negative,,,Negative
33325,2024-06-10,"{'polarity': 0.382, 'neg': 0, 'neu': 0.776, 'p...",0.382,,positive,,Positive


In [91]:
# Convertir la columna 'date' a datetime para asegurarse de que el manejo de fechas sea correcto
df_reduced['date'] = pd.to_datetime(df_reduced['date'])

# Crear un DataFrame de conteos por día y categoría
# counts = df_reduced.groupby(['date', 'Category']).size().unstack(fill_value=0)
counts = df_reduced.groupby('date').size()

In [92]:
counts

date
2020-01-15     1
2020-01-27     1
2020-01-28     1
2020-01-30     2
2020-01-31     1
              ..
2024-06-06    25
2024-06-07    22
2024-06-08     3
2024-06-09     7
2024-06-10     4
Length: 1395, dtype: int64

In [61]:
#Aquí hacemos comprobaciones de los resultados. 
data_on_date = counts.loc['2024-06-07']
print(data_on_date[['Negative', 'Neutro', 'Positive']])

Category
Negative     2
Neutro       5
Positive    15
Name: 2024-06-07 00:00:00, dtype: int64


In [62]:
counts

Category,Negative,Neutro,Positive
date,,,
2020-01-15,0,1,0
2020-01-27,0,0,1
2020-01-28,0,0,1
2020-01-30,0,0,2
2020-01-31,0,0,1
...,...,...,...
2024-06-06,2,3,20
2024-06-07,2,5,15
2024-06-08,0,0,3


In [65]:

# Asegurarse de que la columna 'date' está en formato datetime si aún no lo está
counts.index = pd.to_datetime(counts.index)

# Crear la columna 'Day_of_Week' utilizando el atributo dt.day_name()
counts['Day_of_Week'] = counts.index.day_name()

# Seleccionar solo las columnas deseadas
df_counts = counts[['Day_of_Week', 'Positive', 'Negative', 'Neutro']]

# Mostrar el nuevo DataFrame
print(df_counts)



Category   Day_of_Week  Positive  Negative  Neutro
date                                              
2020-01-15   Wednesday         0         0       1
2020-01-27      Monday         1         0       0
2020-01-28     Tuesday         1         0       0
2020-01-30    Thursday         2         0       0
2020-01-31      Friday         1         0       0
...                ...       ...       ...     ...
2024-06-06    Thursday        20         2       3
2024-06-07      Friday        15         2       5
2024-06-08    Saturday         3         0       0
2024-06-09      Sunday         7         0       0
2024-06-10      Monday         2         1       1

[1395 rows x 4 columns]


In [114]:
df=pd.DataFrame()
df['date']= counts.index

In [115]:
df[['Negative','Positive', 'Neutral']]=df_counts[['Negative','Positive', 'Neutro']]

In [116]:
df['date'].isna().sum()

0

In [29]:
df_trading= pd.read_csv(r'.\data\primary\df_trading.csv')

In [141]:
# volvemos a añadir el día de la semana para poder situarnos con las distintas fechas y miramos los dos dataframes que tenemos para cruzarlos. 

df_trading['date'] = pd.to_datetime(df_trading['Date'], format='ISO8601')

# Cambiar el formato de la columna 'date' a 'dd-mm-yyyy'

df['date'] = pd.to_datetime(df['date'], format='ISO8601')





In [142]:

un_dia=pd.Timedelta(days=1)

In [143]:
df['date'][1]+un_dia

Timestamp('2020-01-28 00:00:00')

In [ ]:
import numpy as np

In [148]:
#se comprueba el número máximo de días en bolsa sin datos
lista=[]
date1=pd.to_datetime('2020-02-13', format='ISO8601')
for date in df_trading['date'].values:
    dif=date-date1
    lista.append(dif)
    date1=date

In [174]:
max(lista)

numpy.timedelta64(345600000000000,'ns')

In [176]:
max_dias=np.timedelta64(max(lista)) / np.timedelta64(1, 'D')

In [177]:
max_dias

4.0

In [171]:
df_trading[['Negative','Positive', 'Neutral']]= 0
for date in df['date'].unique():
    if date in df_trading['date'].values:
        df_trading.loc[df_trading['date'] == date, ['Negative','Positive', 'Neutral']] = df.loc[df['date'] == date, ['Negative','Positive', 'Neutral']].values[0]
    # el número máximo de días sin datos de bolsa es 4
    else:
        for i in range(1,max_dias+1):
            prev_date = date - i * un_dia
            if prev_date in df_trading['date'].values:
                df_trading.loc[df_trading['date'] == prev_date, ['Negative','Positive', 'Neutral']] += df.loc[df['date'] == date, ['Negative','Positive', 'Neutral']].values[0]
                break
                
            if prev_date < df_trading['date'].min():
                break

In [185]:
df_trading.sort_values(by='date').head(20)

,Unnamed: 0,Date,Financial_Sector_Open,Financial_Sector_Close,Volume,VIX_Open,VIX_Close,Historical_Volatility,Alpha,Beta,date,Positive,Negative,Neutral
0,0,2020-02-14,31.100000,31.170000,22080200,14.120000,13.680000,14.021827,0.000733,1.075096,2020-02-14,0,0,0
1,1,2020-02-18,31.080000,30.900000,37768900,14.980000,14.830000,13.901178,0.001108,1.092396,2020-02-18,1,0,0
2,2,2020-02-19,31.010000,31.129999,25643300,14.660000,14.380000,14.055920,0.001325,1.093210,2020-02-19,1,0,0
3,3,2020-02-20,31.020000,31.070000,50344300,14.540000,15.560000,13.920540,0.000971,1.076815,2020-02-20,3,0,0
4,4,2020-02-21,30.920000,30.770000,51128200,17.330000,17.080000,14.111791,0.000615,1.058510,2020-02-21,2,0,0
5,5,2020-02-24,29.799999,29.770000,70455400,22.250000,25.030001,16.927459,-0.000400,1.008691,2020-02-24,0,0,0
6,6,2020-02-25,29.780001,28.760000,111826900,23.280001,27.850000,19.420583,0.000065,1.030153,2020-02-25,4,0,0
7,7,2020-02-26,28.950001,28.549999,95015700,26.629999,27.559999,19.284249,0.000002,1.031710,2020-02-26,0,0,0
8,8,2020-02-27,27.940001,27.330000,143536200,32.139999,39.160000,22.652177,-0.000260,1.010021,2020-02-27,1,0,0
9,9,2020-02-28,26.400000,26.590000,192063500,42.020000,40.110001,23.647517,-0.000530,1.023443,2020-02-28,0,0,0


In [181]:
# elimino las columnas que no interesan:
df_trading_input = df_trading.drop(['Unnamed: 0', 'Date', 'Financial_Sector_Open', 'VIX_Open'], axis=1)

In [184]:
df_trading_input.to_csv(r'.\data\inputs\df_trading_input.csv')


In [186]:
df_trading_input

,Financial_Sector_Close,Volume,VIX_Close,Historical_Volatility,Alpha,Beta,date,Positive,Negative,Neutral
0,31.170000,22080200,13.68,14.021827,0.000733,1.075096,2020-02-14,0,0,0
1,30.900000,37768900,14.83,13.901178,0.001108,1.092396,2020-02-18,1,0,0
2,31.129999,25643300,14.38,14.055920,0.001325,1.093210,2020-02-19,1,0,0
3,31.070000,50344300,15.56,13.920540,0.000971,1.076815,2020-02-20,3,0,0
4,30.770000,51128200,17.08,14.111791,0.000615,1.058510,2020-02-21,2,0,0
...,...,...,...,...,...,...,...,...,...,...
1086,41.320000,31619000,12.74,11.564572,-0.008413,0.690060,2024-06-10,2,1,1
1087,40.849998,54584900,12.85,12.055650,-0.008239,0.719243,2024-06-11,0,0,0
1088,40.810001,46399800,12.04,11.688826,-0.008890,0.704915,2024-06-12,0,0,0
1089,40.779999,39497900,11.94,11.692790,-0.008362,0.725806,2024-06-13,0,0,0
